Take all found IDs and generate a generic dbc assuming 8, 8 bit fields.

In [1]:
import json

In [3]:
import glob
logs = glob.glob("*.log")
logs

['candump-2020-08-06_130024.log',
 'candump-2020-08-06_124354.log',
 'candump-2020-08-06_124435.log',
 'hs_long.log',
 'i_can.log',
 'ican_dump.log',
 'hs_odb_probe.log',
 'lock_unlock.log',
 'candump-2020-08-06_132427.log']

In [4]:
class CanMsg(object):
    def __init__(self, canid, data):
        self.canid=canid
        self.data=data
        
    def __repr__(self):
        return f"CAN<{self.canid}, {self.data}>"
def parse_log(log_file):
    messages = list()
    with open(log_file, "r") as f:
        for can_msg in f.readlines():
            can_id, can_data = can_msg.strip().split(" ")[-1].split("#")
            cm = CanMsg(can_id, can_data)
            messages.append(cm)
    #
    unique_datas = dict()
    #
    for canid in set([msg.canid for msg in messages]):
        id_datas = set([msg.data for msg in messages if msg.canid==canid])
        unique_datas[canid] = id_datas
    #
    return unique_datas

In [5]:
data = parse_log('candump-2020-08-06_132427.log')

In [6]:
data.keys()

dict_keys(['2E0', '108', '230', '581', '03A', '340', '290', '150', '300', '508', '110', '1A8', '070', '080', '250', '2F5', '100', '220', '1B0', '400', '040', '2A7', '2C0', '310', '1A4', '370', '0A0', '361', '241', '360', '2A0', '160', '265', '511', '020', '1E0', '240', '3A0', '415', '320', '504', '541', '060', '255', '40A', '030', '1A0', '090', '1C0', '2B0', '270', '501', '083', '405'])

In [7]:
def gen_dbc_entry(can_id):
    dbc_entry = {'id': int(f"{can_id}", 16),
     'is_extended_frame': False,
     'name': f'Ox{can_id}',
     'signals': list()}

    def gen_sig(start_bit, bit_length=8):
        return {'bit_length': bit_length,
         'factor': "1",
         'is_big_endian': True,
         'is_float': False,
         'is_signed': False,
         'name': f'Ox{can_id}_{start_bit}',
         'offset': "0",
         'start_bit': start_bit
        }
    
    for start_bit in range(0, 64, 8):
        sig = gen_sig(start_bit=start_bit)
        dbc_entry["signals"].append(sig)
        
    return dbc_entry

In [9]:
dbc = {
    "messages": list()
}

unique_message_dbc_threshold = 5

for can_id, can_messages in data.items():
    if len(can_messages)< unique_message_dbc_threshold:
        continue
    dbc["messages"].append(gen_dbc_entry(can_id))

with open("spewed_dbc.json", "w") as fp:
    json.dump(obj=dbc, fp=fp, indent=4, sort_keys=True)